In [30]:
# 크롤링을 위한 셀레니움
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 데이터 프레임 작성을 위한 판다스
import pandas as pd

# 대기시간을 휘한 라이브러리
import random
import time

from selenium.webdriver.common.action_chains import ActionChains

def croll():
    for x in range(1, 101):
        try:
            # 링크를 새 탭에서 열기
            link = driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[3]/div[2]/ul/li[{x}]/div[2]/p[1]/a')
            actions = ActionChains(driver)
            actions.key_down(Keys.COMMAND).click(link).key_up(Keys.COMMAND).perform()  # 새 탭에서 열기

            driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
            time.sleep(5)
        except:
            break

        try:
            title = driver.find_element(By.CSS_SELECTOR, '#thesisInfoDiv > div.thesisInfoTop > h3').text
        except:
            title = None

        try:
            date  = driver.find_element(By.CSS_SELECTOR, '#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(5) > div > p').text
        except:
            date = None

        try:
            keyword  = driver.find_element(By.CSS_SELECTOR, '#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(7) > div > p').text
        except:
            keyword = None
        
        try:
            abstract_text = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > p').text
            if abstract_text == '국문 초록 (Abstract)':
                driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > a').click()
                time.sleep(0.2)
                abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(5) > p').text
                try:
                    driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > a').click()
                    time.sleep(0.2)
                    multi_abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(5) > p').text
                except:
                    multi_abstract = None
            elif abstract_text == '다국어 초록 (Multilingual Abstract)':
                driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > a').click()
                time.sleep(0.2)
                multi_abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(5) > p').text
                try:
                    driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > a').click()
                    time.sleep(0.2)
                    abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(5) > p').text
                except:
                    abstract = None
            else:
                abstract = None
                multi_abstract = None
        except:
            abstract = None
            multi_abstract = None
        
        titles.append(title)
        dates.append(date)
        keywords.append(keyword)
        abstracts.append(abstract)
        multi_abstracts.append(multi_abstract)
        
        driver.close()  # 새 탭을 닫고
        driver.switch_to.window(driver.window_handles[0])  # 첫 번째 탭(원래 페이지)로 돌아감
        time.sleep(5)

# 크롤링한 논문 설정
search = input('검색할 단어를 입력 : ')
year = input('지금으로부터 몇년 전의 논문을 가져올지 입력 : ')
time.sleep(5)

# 크롬 드라이버를 실행하고 메인페이지 실행
driver = webdriver.Chrome()
driver.get('https://www.riss.kr/index.do')
time.sleep(30)

# 원하는 키워드로 논문을 검색
driver.find_element(By.XPATH, '//*[@id="query"]').click()
driver.find_element(By.XPATH, '//*[@id="query"]').send_keys(search)
driver.find_element(By.CSS_SELECTOR, '#in > div > button').click()
time.sleep(10)

# 학술논문 페이지로 넘어가기
driver.find_element(By.CSS_SELECTOR, '#divContent > div > div > div.srchResultW.totalSrch > div:nth-child(1) > div.topRight > a > img').click()
time.sleep(10)

# 년도 설정
driver.find_element(By.CSS_SELECTOR, f'#mCSB_7_container > li:nth-child({year}) > label > span.checkBox').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, f'#mCSB_7_container > li:nth-child({year}) > a').click()
time.sleep(10)

# 논문을 100개씩 보기로 설정
driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/div[3]/div[1]/label').click()
time.sleep(5)
driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/div[3]/div[2]/div/ul/li[5]/a').click()
time.sleep(5)
driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/button').click()
time.sleep(5)

# 페이지 수 계산
thesis_num = driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[1]/dl/dd/span/span').text
thesis_num = int(thesis_num[:].replace(',',''))
page_num = thesis_num // 100 + 1
print('추출할 논문의 수 : ',thesis_num)

# 자료들을 모아둘 리스트 선언
titles = []
dates = []
keywords=[]
abstracts = []
multi_abstracts = []

# 첫 페이지 크롤링
croll()

# 나머지 페이지 크롤링
for p in range(2, page_num+1):
    try:
        if p <= 10:
            driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[4]/a[{p+1}]').click()
        else:
            if p == 11:
                driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[12]').click()
            elif p % 10 == 1:
                driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[13]').click()
            else:
                if p % 10 == 0:
                    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[12]').click()
                else:
                    driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[4]/a[{p%10+2}]').click()
        time.sleep(3)
        if p % 3 == 0:
            time.sleep(random.randint(5,7))
        croll()
    except:
        break

# 크롤링한 데이터로 데이터 프레임 생성
data = pd.DataFrame({
    'title': titles,
    'date': dates,
    'keyword':keywords,
    'abstract': abstracts,
    'multilingual': multi_abstracts
})

# 데이터 프레임 저장
csv_file_path = f'{search}_{year}_riss_academic.csv'
data.to_csv(csv_file_path, index=False)

# 드라이버 종료
driver.close()

print('추출한 논문의 수 : ', len(titles))

추출할 논문의 수 :  1558


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000101376138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x000000010136e988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x0000000100dd47d8 cxxbridge1$string$len + 88828
3   chromedriver                        0x0000000100e0d304 cxxbridge1$string$len + 321064
4   chromedriver                        0x0000000100e0e1b4 cxxbridge1$string$len + 324824
5   chromedriver                        0x0000000101340fc0 cxxbridge1$str$ptr + 3436424
6   chromedriver                        0x00000001013442dc cxxbridge1$str$ptr + 3449508
7   chromedriver                        0x0000000101327e60 cxxbridge1$str$ptr + 3333672
8   chromedriver                        0x0000000101344b9c cxxbridge1$str$ptr + 3451748
9   chromedriver                        0x0000000101319678 cxxbridge1$str$ptr + 3274304
10  chromedriver                        0x000000

In [ ]:
# 크롤링을 위한 셀레니움
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 데이터 프레임 작성을 위한 판다스
import pandas as pd

# 대기시간을 휘한 라이브러리
import random
import time

from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome()
driver.get('https://www.riss.kr/index.do')
time.sleep(30)

In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import random
import time
from selenium.webdriver.common.action_chains import ActionChains

# 크롤링 함수
def crawl_page(driver, page_num):
    titles = []
    dates = []
    keywords = []
    abstracts = []
    multi_abstracts = []
    
    for x in range(1, 101):
        try:
            # 링크를 새 탭에서 열기
            link = driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[3]/div[2]/ul/li[{x}]/div[2]/p[1]/a')
            actions = ActionChains(driver)
            actions.key_down(Keys.COMMAND).click(link).key_up(Keys.COMMAND).perform()  # 새 탭에서 열기
            driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
            time.sleep(5)
        except:
            break

        # 데이터 추출 함수
        title = get_element_text(driver, '#thesisInfoDiv > div.thesisInfoTop > h3')
        date = get_element_text(driver, '#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(5) > div > p')
        keyword = get_element_text(driver, '#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(7) > div > p')
        abstract, multi_abstract = get_abstracts(driver)
        
        titles.append(title)
        dates.append(date)
        keywords.append(keyword)
        abstracts.append(abstract)
        multi_abstracts.append(multi_abstract)
        
        driver.close()  # 새 탭을 닫고
        driver.switch_to.window(driver.window_handles[0])  # 첫 번째 탭(원래 페이지)로 돌아감
        time.sleep(5)
        
    return titles, dates, keywords, abstracts, multi_abstracts

# 엘리먼트에서 텍스트 추출 (None 처리)
def get_element_text(driver, css_selector):
    try:
        return driver.find_element(By.CSS_SELECTOR, css_selector).text
    except:
        return None

# 초록 정보 추출
def get_abstracts(driver):
    abstract, multi_abstract = None, None
    try:
        abstract_text = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > p').text
        if abstract_text == '국문 초록 (Abstract)':
            driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > a').click()
            time.sleep(0.2)
            abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(5) > p').text
            try:
                driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > a').click()
                time.sleep(0.2)
                multi_abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(5) > p').text
            except:
                multi_abstract = None
        elif abstract_text == '다국어 초록 (Multilingual Abstract)':
            driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > a').click()
            time.sleep(0.2)
            multi_abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(5) > p').text
            try:
                driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > a').click()
                time.sleep(0.2)
                abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(5) > p').text
            except:
                abstract = None
    except:
        pass
    return abstract, multi_abstract

# 페이지 전환 함수
def go_to_page(driver, page_num):
    if page_num <= 10:
        driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[4]/a[{page_num+1}]').click()
    else:
        if page_num == 11:
            driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[12]').click()
        elif page_num % 10 == 1:
            driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[13]').click()
        else:
            if page_num % 10 == 0:
                driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[12]').click()
            else:
                driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[4]/a[{page_num % 10 + 2}]').click()
    time.sleep(3)

# 크롬 드라이버 시작 및 메인 페이지 접근
def init_driver():
    driver = webdriver.Chrome()
    driver.get('https://www.riss.kr/index.do')
    time.sleep(30)
    return driver

# 메인 크롤링 과정
def main_crawl():
    search = input('검색할 단어를 입력 : ')
    year = input('지금으로부터 몇년 전의 논문을 가져올지 입력 : ')
    time.sleep(5)

    driver = init_driver()

    # 키워드 검색
    driver.find_element(By.XPATH, '//*[@id="query"]').click()
    driver.find_element(By.XPATH, '//*[@id="query"]').send_keys(search)
    driver.find_element(By.CSS_SELECTOR, '#in > div > button').click()
    time.sleep(10)

    # 논문 페이지로 이동
    driver.find_element(By.CSS_SELECTOR, '#divContent > div > div > div.srchResultW.totalSrch > div:nth-child(1) > div.topRight > a > img').click()
    time.sleep(10)

    # 년도 설정
    driver.find_element(By.CSS_SELECTOR, f'#mCSB_7_container > li:nth-child({year}) > label > span.checkBox').click()
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR, f'#mCSB_7_container > li:nth-child({year}) > a').click()
    time.sleep(10)

    # 논문 100개씩 보기
    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/div[3]/div[1]/label').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/div[3]/div[2]/div/ul/li[5]/a').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/button').click()
    time.sleep(5)

    # 논문 수와 페이지 수 계산
    thesis_num = driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[1]/dl/dd/span/span').text
    thesis_num = int(thesis_num[:].replace(',', ''))
    page_num = thesis_num // 100 + 1
    print('추출할 논문의 수 : ', thesis_num)

    # 첫 번째 페이지 크롤링
    titles, dates, keywords, abstracts, multi_abstracts = crawl_page(driver, 1)

    # 나머지 페이지 크롤링
    for p in range(2, page_num + 1):
        try:
            go_to_page(driver, p)
            if p % 3 == 0:
                time.sleep(random.randint(5, 7))
            page_titles, page_dates, page_keywords, page_abstracts, page_multi_abstracts = crawl_page(driver, p)
            titles.extend(page_titles)
            dates.extend(page_dates)
            keywords.extend(page_keywords)
            abstracts.extend(page_abstracts)
            multi_abstracts.extend(page_multi_abstracts)
        except:
            break

    # 데이터 프레임 생성 및 저장
    data = pd.DataFrame({
        'title': titles,
        'date': dates,
        'keyword': keywords,
        'abstract': abstracts,
        'multilingual': multi_abstracts
    })

    csv_file_path = f'{search}_{year}_riss_academic.csv'
    data.to_csv(csv_file_path, index=False)

    # 드라이버 종료
    driver.close()

    print('추출한 논문의 수 : ', len(titles))

if __name__ == "__main__":
    main_crawl()



추출할 논문의 수 :  1558


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000102bb2138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000102baa988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x00000001026107d8 cxxbridge1$string$len + 88828
3   chromedriver                        0x0000000102649304 cxxbridge1$string$len + 321064
4   chromedriver                        0x000000010264a1b4 cxxbridge1$string$len + 324824
5   chromedriver                        0x0000000102b7cfc0 cxxbridge1$str$ptr + 3436424
6   chromedriver                        0x0000000102b802dc cxxbridge1$str$ptr + 3449508
7   chromedriver                        0x0000000102b63e60 cxxbridge1$str$ptr + 3333672
8   chromedriver                        0x0000000102b80b9c cxxbridge1$str$ptr + 3451748
9   chromedriver                        0x0000000102b55678 cxxbridge1$str$ptr + 3274304
10  chromedriver                        0x000000

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import random
import time
from selenium.webdriver.common.action_chains import ActionChains

# 크롤링 함수
def crawl_page(driver, page_num):
    titles = []
    dates = []
    keywords = []
    abstracts = []
    multi_abstracts = []
    
    # 페이지 끝까지 스크롤
    scroll_to_bottom(driver)
    
    for x in range(1, 101):
        try:
            # 링크를 새 탭에서 열기
            link = driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[3]/div[2]/ul/li[{x}]/div[2]/p[1]/a')
            actions = ActionChains(driver)
            actions.key_down(Keys.COMMAND).click(link).key_up(Keys.COMMAND).perform()  # 새 탭에서 열기
            driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
            time.sleep(5)
        except:
            break

        # 데이터 추출 함수
        title = get_element_text(driver, '#thesisInfoDiv > div.thesisInfoTop > h3')
        date = get_element_text(driver, '#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(5) > div > p')
        keyword = get_element_text(driver, '#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(7) > div > p')
        abstract, multi_abstract = get_abstracts(driver)
        
        titles.append(title)
        dates.append(date)
        keywords.append(keyword)
        abstracts.append(abstract)
        multi_abstracts.append(multi_abstract)
        
        driver.close()  # 새 탭을 닫고
        driver.switch_to.window(driver.window_handles[0])  # 첫 번째 탭(원래 페이지)로 돌아감
        time.sleep(5)
        
    return titles, dates, keywords, abstracts, multi_abstracts

# 엘리먼트에서 텍스트 추출 (None 처리)
def get_element_text(driver, css_selector):
    try:
        return driver.find_element(By.CSS_SELECTOR, css_selector).text
    except:
        return None

# 초록 정보 추출
def get_abstracts(driver):
    abstract, multi_abstract = None, None
    try:
        abstract_text = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > p').text
        if abstract_text == '국문 초록 (Abstract)':
            driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > a').click()
            time.sleep(0.2)
            abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(5) > p').text
            try:
                driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > a').click()
                time.sleep(0.2)
                multi_abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(5) > p').text
            except:
                multi_abstract = None
        elif abstract_text == '다국어 초록 (Multilingual Abstract)':
            driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > a').click()
            time.sleep(0.2)
            multi_abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(5) > p').text
            try:
                driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > a').click()
                time.sleep(0.2)
                abstract = driver.find_element(By.CSS_SELECTOR, '#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(5) > p').text
            except:
                abstract = None
    except:
        pass
    return abstract, multi_abstract

# 페이지 전환 함수
def go_to_page(driver, page_num):
    if page_num <= 10:
        driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[4]/a[{page_num+1}]').click()
    else:
        if page_num == 11:
            driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[12]').click()
        elif page_num % 10 == 1:
            driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[13]').click()
        else:
            if page_num % 10 == 0:
                driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[4]/a[12]').click()
            else:
                driver.find_element(By.XPATH, f'//*[@id="divContent"]/div/div[2]/div/div[4]/a[{page_num % 10 + 2}]').click()
    time.sleep(3)

# 페이지 끝까지 스크롤
def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 스크롤 후 기다리기

# 크롬 드라이버 시작 및 메인 페이지 접근
def init_driver():
    driver = webdriver.Chrome()
    driver.get('https://www.riss.kr/index.do')
    time.sleep(30)
    return driver

# 메인 크롤링 과정
def main_crawl():
    search = input('검색할 단어를 입력 : ')
    year = input('지금으로부터 몇년 전의 논문을 가져올지 입력 : ')
    time.sleep(5)

    driver = init_driver()

    # 키워드 검색
    driver.find_element(By.XPATH, '//*[@id="query"]').click()
    driver.find_element(By.XPATH, '//*[@id="query"]').send_keys(search)
    driver.find_element(By.CSS_SELECTOR, '#in > div > button').click()
    time.sleep(10)

    # 논문 페이지로 이동
    driver.find_element(By.CSS_SELECTOR, '#divContent > div > div > div.srchResultW.totalSrch > div:nth-child(1) > div.topRight > a > img').click()
    time.sleep(10)

    # 년도 설정
    driver.find_element(By.CSS_SELECTOR, f'#mCSB_7_container > li:nth-child({year}) > label > span.checkBox').click()
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR, f'#mCSB_7_container > li:nth-child({year}) > a').click()
    time.sleep(10)

    # 논문 100개씩 보기
    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/div[3]/div[1]/label').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/div[3]/div[2]/div/ul/li[5]/a').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[2]/button').click()
    time.sleep(5)

    # 논문 수와 페이지 수 계산
    thesis_num = driver.find_element(By.XPATH, '//*[@id="divContent"]/div/div[2]/div/div[1]/dl/dd/span/span').text
    thesis_num = int(thesis_num[:].replace(',', ''))
    page_num = thesis_num // 100 + 1
    print('추출할 논문의 수 : ', thesis_num)

    # 첫 번째 페이지 크롤링
    titles, dates, keywords, abstracts, multi_abstracts = crawl_page(driver, 1)

    # 나머지 페이지 크롤링
    for p in range(2, page_num + 1):
        try:
            go_to_page(driver, p)
            if p % 3 == 0:
                time.sleep(random.randint(5, 7))
            page_titles, page_dates, page_keywords, page_abstracts, page_multi_abstracts = crawl_page(driver, p)
            titles.extend(page_titles)
            dates.extend(page_dates)
            keywords.extend(page_keywords)
            abstracts.extend(page_abstracts)
            multi_abstracts.extend(page_multi_abstracts)
        except:
            break

    # 데이터 프레임 생성 및 저장
    data = pd.DataFrame({
        'title': titles,
        'date': dates,
        'keyword': keywords,
        'abstract': abstracts,
        'multilingual': multi_abstracts
    })

    csv_file_path = f'{search}_{year}_riss.csv'
    data.to_csv(csv_file_path, encoding='utf-8-sig', index=False)
    print(f'파일 저장 완료: {csv_file_path}')

    driver.quit()

if __name__ == '__main__':
    main_crawl()


추출할 논문의 수 :  1558


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000100b62138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000100b5a988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x00000001005c07d8 cxxbridge1$string$len + 88828
3   chromedriver                        0x00000001005f9304 cxxbridge1$string$len + 321064
4   chromedriver                        0x00000001005fa1b4 cxxbridge1$string$len + 324824
5   chromedriver                        0x0000000100b2cfc0 cxxbridge1$str$ptr + 3436424
6   chromedriver                        0x0000000100b302dc cxxbridge1$str$ptr + 3449508
7   chromedriver                        0x0000000100b13e60 cxxbridge1$str$ptr + 3333672
8   chromedriver                        0x0000000100b30b9c cxxbridge1$str$ptr + 3451748
9   chromedriver                        0x0000000100b05678 cxxbridge1$str$ptr + 3274304
10  chromedriver                        0x000000